# Tarefa 3: Usar o Amazon Bedrock para responder perguntas

Neste caderno, você aprenderá a usar o modelo Titan do Bedrock para fornecer respostas informativas a consultas enviando a solicitação com todo o contexto relevante ao modelo e esperando a resposta, enfrentando o desafio de fazer com que o modelo retorne respostas factuais para perguntas sem precisar preparar e indexar documentos previamente.

Este caderno simula o que a **Geração aumentada de recuperação (RAG)** faria, mas sem usar RAG de fato. Essa abordagem funciona para documentos curtos ou aplicações singleton. Sua escala pode não ser adequada no caso de uso de responder perguntas em nível empresarial, visto que nem todos os documentos grandes necessários cabem no prompt enviado para o modelo.

**Responder perguntas (QA)** é uma tarefa importante que envolve extrair respostas para consultas factuais feitas em uma linguagem natural. Normalmente, um sistema de QA processa uma consulta em uma base de conhecimento que contém dados estruturados ou não estruturados e gera uma resposta com informações precisas. Garantir um alto nível de acurácia é essencial para desenvolver um sistema de resposta a perguntas útil e confiável, principalmente para casos de uso empresariais.


## Cenário

Você tenta modelar uma situação na AnyCompany em que pede para que um modelo de respostas a perguntas dê informações sobre a troca de pneus para um modelo de veículo específico que ela fabrica. Primeiro, você consulta o modelo usando uma abordagem "zero-shot" para ver se ele pode dar respostas relevantes com base apenas nos dados de treinamento dele.

No entanto, você percebe que o modelo parece estar "alucinando" respostas mais genéricas, conforme evidenciado quando você testa um modelo de veículo falso e obtém respostas semelhantes. Isso sugere a necessidade de aumentar o treinamento do modelo com os manuais de veículos reais da Example Company para dar detalhes sobre os pneus de cada modelo.

Neste laboratório, você simula essa abordagem de "geração aumentada de recuperação" (RAG) sem dados externos. Você fornece um trecho detalhado do manual explicando como trocar os pneus no veículo Modelo Z da AnyCompany. Você testa se o modelo agora pode fornecer uma resposta personalizada e precisa usando esse exemplo de conteúdo contextualizado.

## Tarefa 3.1: Configurar o ambiente

Nesta tarefa, você vai configurar o ambiente.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tarefa 3.2: Perguntas e respostas com o conhecimento do modelo
Nesta seção, tentamos usar modelos oferecidos pelo serviço Bedrock para responder a perguntas com base no conhecimento que eles adquiriram durante a fase de treinamento.

Nessa tarefa, você usa o método invoke_model () do cliente do Amazon Bedrock. Os parâmetros obrigatórios para usar esse método são: "modelId", que representa o ARN do modelo do Amazon Bedrock; além de "body", que é o prompt de sua tarefa.

O formato do texto do prompt muda conforme o provedor do modelo de base selecionado. Você verá isso em detalhes abaixo.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Você tenta usar modelos fornecidos pelo serviço Bedrock para responder perguntas com base no conhecimento adquirido durante a fase de treinamento.

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## Tarefa 3.3: Invocar o modelo ao usar o formato de texto JSON para gerar a resposta

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


O modelo fornece uma resposta descrevendo o processo de troca do pneu furado do carro, mas a mesma explicação pode ser válida para qualquer carro. Infelizmente, essa não é a resposta correta para um AnyCompany AC8, que não tem estepe. Isso ocorre porque o modelo foi treinado com dados que contêm instruções sobre a troca de pneus em carros.

Outro exemplo desse problema pode ser encontrado ao tentar fazer a mesma pergunta usando uma marca e modelo de carro totalmente falsos, como Amazon Tirana.

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

Considerando a pergunta do prompt, o modelo é incapaz de fornecer uma resposta realista.

Para resolver esse problema e fazer com que o modelo dê respostas com base nas instruções específicas válidas para o modelo do seu carro, você pode aumentar imediatamente o conhecimento do modelo fornecendo uma base de conhecimento adicional como parte do prompt.

Vamos ver como você pode usar isso para melhorar sua aplicação.

Suponha que o trecho a seguir seja do manual do AnyCompany AC8 (na realidade, não é o manual real, mas trate-o como tal). Esse documento também é convenientemente curto o bastante para caber inteiramente na janela de contexto do Titan Large.

```plain
Pneus e pressão dos pneus:

Os pneus são feitos de borracha preta e são montados nas rodas do seu veículo. Eles oferecem a aderência necessária para dirigir, manobrar e frear. Dois fatores importantes a considerar são a pressão e o desgaste dos pneus, pois eles podem afetar o desempenho e dirigibilidade do carro.

Onde encontrar a pressão recomendada para os pneus:

Você pode encontrar as especificações de pressão recomendada para o pneu na etiqueta de calibragem localizada na coluna B do lado do motorista no veículo. Como alternativa, você pode consultar o manual do veículo para encontrar esta informação. A pressão recomendada para o pneu pode variar dependendo da velocidade e do número de ocupantes ou carga máxima dentro do veículo.

Recalibragem dos pneus:

É importante verificar a pressão dos pneus quando eles estiverem frios. Para isso, o veículo deve permanecer parado por pelo menos três horas. Isso garante que a temperatura dos pneus se iguale à ambiente.

Para recalibrar os pneus:

    Verifique a pressão recomendada para os pneus do seu veículo.
    Siga as instruções fornecidas na bomba pneumática e calibre um ou mais pneus com a pressão correta.
    No painel central do veículo, abra a aplicação “Status do carro”.
    Navegue até a aba “Pressão dos pneus”.
    Pressione a opção “Calibrar pressão” e confirme a ação.
    Dirija o carro por alguns minutos a uma velocidade acima de 30 km/h para calibrar a pressão dos pneus.

Observação: em alguns casos, pode ser necessário dirigir por mais de 15 minutos para apagar qualquer símbolo ou mensagem de aviso relacionado à pressão do pneu. Se os avisos persistirem, deixe os pneus esfriarem e repita as etapas acima.

Pneu furado:

Se algum pneu furar enquanto você dirige, é possível vedar o furo de forma temporária e recalibrar o pneu usando um kit de reparo. Normalmente, esse kit é armazenado abaixo do forro no porta-malas do veículo.

Instruções para utilizar o kit de reparo para pneu:

    Abra a tampa traseira ou o porta-malas do veículo.
    Levante o forro do porta-malas para acessar o kit de reparo para pneu.
    Siga as instruções descritas no kit de reparo para vedar o furo no pneu.
    Após usar o kit, coloque-o de volta no lugar original de forma segura.
    Entre em contato com a Rivesla ou com um serviço apropriado para receber ajuda no descarte e substituição da garrafa de vedante usada.

Lembre-se de que p kit de reparo para pneu é uma solução temporária. Ele permite que você dirija por no máximo 10 minutos ou 8 km (o que acontecer primeiro) a uma velocidade máxima de 80 km/h. Aconselhamos que você troque o pneu furado ou encontre um profissional para repará-lo o mais rápido possível.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### Agora, transmita o trecho inteiro para o modelo com a pergunta.

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### Tarefa 3.4: Invocar o modelo via boto3 para gerar a resposta

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

Como o modelo demora um pouco para compreender o contexto e gerar uma resposta relevante para você, isso pode resultar em uma experiência do usuário ruim, pois é necessário aguardar uma resposta por alguns segundos.

O Bedrock também é compatível com o recurso de transmissão em que o serviço gera saída conforme o modelo gera tokens. Aqui, é possível ver um exemplo de como você pode implementar isso.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

A resposta fornece instruções passo a passo resumidas sobre como trocar os pneus. 

Você aprendeu a usar a geração aumentada de recuperação (RAG) ou o processo de aumento para gerar uma resposta com curadoria adaptada ao contexto específico e às informações fornecidas.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 4**.